In [1]:
from setup import general
from setup import stat
from setup import r

import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import pingouin as pg
from pingouin import ancova # anconva
from pingouin import multivariate_normality # for pearson correlation
from rpy2 import robjects

In [2]:
general.pd_setup()

/home/jzni/anaconda3/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.12, the latest is 0.4.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [3]:
%store -r Scale_pt Scale_hc nreps Con Pt_included

In [4]:
general.fill_na_mean(Scale_pt)
scales = Scale_pt.columns[1:]

# Analysis
健康人与患者前测的行为评分均值比较

In [5]:

hc_pt =[]
for scale in scales:
    lev = stats.levene(Scale_hc[scale],Scale_pt[scale])
    equal_var = True if lev[1]>0.05 else False
    t_result = stats.ttest_ind(Scale_hc[scale],Scale_pt[scale], equal_var=equal_var)
    
    perm_ind_t_p = robjects.r['perm_ind_t'](r.to_csv_r(Scale_hc, 'hc', scale), r.to_csv_r(Scale_pt, 'pt', scale), 3 + len(scale), nreps=nreps)
    
    # 将年龄作为协变量
    ancova_a = Scale_hc[[scale]]; ancova_a.loc[:,'Type'] = [1 for i in range(ancova_a.shape[0])]
    ancova_b = Scale_pt[[scale]]; ancova_b.loc[:,'Type'] = [2 for i in range(ancova_b.shape[0])]
    ancova_df = pd.concat([ancova_a,ancova_b])
    ancova_age_1 = general.get_control_info(Con,ancova_df, 'Age');ancova_age_2 = general.get_patient_info(Pt_included,ancova_df, 'Age')
    ancova_age = pd.DataFrame(pd.concat([ancova_age_1,ancova_age_2]))
    ancova_df = ancova_df.join(ancova_age)
    anc = ancova(data=ancova_df, dv=str(scale), between='Type', covar='Age')
    
    res = list(stats.shapiro(Scale_hc[scale])+stats.shapiro(Scale_pt[scale])+lev+tuple([round(t_result[0],5),round(t_result[1],5)])+tuple(perm_ind_t_p))
    res.append(anc.loc[0,'p-unc'])
    hc_pt.append(res)
hc_pt_df = pd.DataFrame(hc_pt, index=scales, columns=['hc_norm_stat','hc_norm_p', 'pt_norm_stat','pt_norm_p','levene_stat', 'levene_p','ind_t','ind_t_p','perm_ind_t_p','ancova_p'])


/home/jzni/anaconda3/lib/python3.8/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
/home/jzni/anaconda3/lib/python3.8/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
/home/jzni/anaconda3/lib/python3.8/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


In [6]:
hc_pt_df

,hc_norm_stat,hc_norm_p,pt_norm_stat,pt_norm_p,levene_stat,levene_p,ind_t,ind_t_p,perm_ind_t_p,ancova_p
VAS,1.0000,1.0000,0.9469,0.4090,27.3496,0.0000,-8.8352,0.0000,0.0000,0.0000
SFMPQ,1.0000,1.0000,0.9835,0.9823,32.2103,0.0000,-8.9150,0.0000,0.0000,0.0000
PDQ,1.0000,1.0000,0.9242,0.1738,20.0806,0.0001,-7.9385,0.0000,0.0000,0.0000
PSQ,0.9613,0.6267,0.9618,0.6654,3.5501,0.0684,0.8064,0.4258,0.4254,0.4267
FPQ_Severe,0.8910,0.0401,0.9295,0.2136,0.5345,0.4699,1.9418,0.0607,0.0650,0.0792
FPQ_Mild,0.9359,0.2467,0.9089,0.0957,1.1744,0.2864,1.7362,0.0919,0.0992,0.1131
FPQ_Medical,0.9664,0.7272,0.8493,0.0105,1.3149,0.2598,1.3513,0.1858,0.1998,0.2325
FPQ_Total,0.9560,0.5259,0.9536,0.5160,0.0223,0.8821,1.8293,0.0764,0.0816,0.0990
PCS_1,0.9417,0.3097,0.8435,0.0085,1.5025,0.2290,-0.6378,0.5280,0.5236,0.5994
PCS_2,0.8671,0.0159,0.8982,0.0633,1.6233,0.2115,-0.1898,0.8507,0.8540,0.7848


In [7]:
hc_pt_result = general.create_final_table(hc_pt_df, Scale_hc, Scale_pt, columns=['healthy control', 'patient'])
%store hc_pt_result
hc_pt_result

Stored 'hc_pt_result' (DataFrame)


,healthy control,patient,t value,p value,ancova p value
VAS,0.0 ± 0.0,56.25 ± 26.25,-8.8400,0.0000,0.0000
SFMPQ,0.0 ± 0.0,73.06 ± 33.79,-8.9100,0.0000,0.0000
PDQ,0.0 ± 0.0,16.76 ± 8.71,-7.9400,0.0000,0.0000
PSQ,74.11 ± 17.18,67.53 ± 29.77,0.8100,0.4258,0.4267
FPQ_Severe,36.83 ± 8.53,30.42 ± 10.93,1.9400,0.0650,0.0792
FPQ_Mild,23.56 ± 7.71,19.33 ± 6.6,1.7400,0.0919,0.1131
FPQ_Medical,27.72 ± 9.71,23.5 ± 8.71,1.3500,0.1998,0.2325
FPQ_Total,88.17 ± 23.43,73.25 ± 24.82,1.8300,0.0764,0.0990
PCS_1,5.5 ± 3.42,6.17 ± 2.71,-0.6400,0.5236,0.5994
PCS_2,3.22 ± 3.37,3.42 ± 2.66,-0.1900,0.8540,0.7848


结果包括shapiro正态性检验、levene方差齐性检验、两独立样本t检验、两独立样本置换t检验、协方差分析（年龄为协变量）

健康人与患者对比有显著差异的评分：
* SAI
* HAMA
* BDI
  
边缘显著的评分：
* FPQ 严重疼痛 及 总分
* PVAQ
* HAMD （协方差不显著）

大五人格的结果，患者与健康人神经质评分有显著差异，与焦虑量表的显著差异一致。